# Notebook 1 - Preprocessing & Integrity Checks

# 0. Import libraries and load data

In [33]:
import pandas as pd
from pathlib import Path

current_dir = Path().resolve()
root = current_dir.parents[1]  # Sube de notebooks/ a GridSense/

data_dir = root / 'data' / 'raw_data'

df_openmeteo = pd.read_csv(data_dir / 'openmeteo_all_cities.csv', parse_dates=['time'])
df_pvgis = pd.read_csv(data_dir / 'pvgis_all_cities.csv', parse_dates=['time'])




# 1. Check dimensions and columns

In [34]:
print("Open-Meteo shape:", df_openmeteo.shape)
print("PVGIS shape:", df_pvgis.shape)

print("Open-Meteo columns:", df_openmeteo.columns.tolist())
print("PVGIS columns:", df_pvgis.columns.tolist())

Open-Meteo shape: (7293312, 10)
PVGIS shape: (7293312, 3)
Open-Meteo columns: ['time', 'temperature_2m', 'cloudcover', 'windspeed_10m', 'winddirection_10m', 'shortwave_radiation', 'direct_radiation', 'diffuse_radiation', 'cloud_cover', 'city']
PVGIS columns: ['time', 'global_irradiance_W_m2', 'city']


# 2. Check for duplicates

In [35]:
print("Duplicados Open-Meteo:", df_openmeteo.duplicated(subset=["time", "city"]).sum())
print("Duplicados PVGIS:", df_pvgis.duplicated(subset=["time", "city"]).sum())


Duplicados Open-Meteo: 0
Duplicados PVGIS: 0


# 3. Check for missing values

In [36]:
print("Missing values in Open-Meteo (%):\n", df_openmeteo.isnull().mean() * 100)
print("\nMissing values in PVGIS (%):\n", df_pvgis.isnull().mean() * 100)


Missing values in Open-Meteo (%):
 time                   0.0
temperature_2m         0.0
cloudcover             0.0
windspeed_10m          0.0
winddirection_10m      0.0
shortwave_radiation    0.0
direct_radiation       0.0
diffuse_radiation      0.0
cloud_cover            0.0
city                   0.0
dtype: float64

Missing values in PVGIS (%):
 time                      0.0
global_irradiance_W_m2    0.0
city                      0.0
dtype: float64


# 4. Verify time coverage

In [37]:
# Align PVGIS time to the start of the hour
df_pvgis['time'] = pd.to_datetime(df_pvgis['time']) - pd.to_timedelta(9, unit='m')
df_pvgis['time'] = df_pvgis['time'].dt.floor('h')

# Clip PVGIS time to match Open-Meteo range
min_time = df_openmeteo['time'].min()
max_time = df_openmeteo['time'].max()

df_pvgis = df_pvgis[(df_pvgis['time'] >= min_time) & (df_pvgis['time'] <= max_time)]

# Print new ranges
print("Open-Meteo time range:", df_openmeteo['time'].min(), "-", df_openmeteo['time'].max())
print("PVGIS time range:", df_pvgis['time'].min(), "-", df_pvgis['time'].max())





Open-Meteo time range: 2005-01-01 00:00:00 - 2020-12-31 23:00:00
PVGIS time range: 2005-01-01 00:00:00 - 2020-12-31 23:00:00


# 5. Check temporal consistency (missing timestamps)

In [38]:
cities = df_openmeteo['city'].unique()

for city in cities:
    df_city = df_openmeteo[df_openmeteo['city'] == city].set_index('time').sort_index()
    expected = pd.date_range(df_city.index.min(), df_city.index.max(), freq='H')
    missing = expected.difference(df_city.index)
    print(f"{city}: {len(missing)} missing hours in Open-Meteo.")


C:\Users\mmarc\AppData\Local\Temp\ipykernel_7360\449978571.py:5: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  expected = pd.date_range(df_city.index.min(), df_city.index.max(), freq='H')


alava: 0 missing hours in Open-Meteo.
albacete: 0 missing hours in Open-Meteo.
alicante: 0 missing hours in Open-Meteo.
almeria: 0 missing hours in Open-Meteo.
asturias: 0 missing hours in Open-Meteo.
avila: 0 missing hours in Open-Meteo.
badajoz: 0 missing hours in Open-Meteo.
barcelona: 0 missing hours in Open-Meteo.
burgos: 0 missing hours in Open-Meteo.
caceres: 0 missing hours in Open-Meteo.
cadiz: 0 missing hours in Open-Meteo.
cantabria: 0 missing hours in Open-Meteo.
castellon: 0 missing hours in Open-Meteo.
ceuta: 0 missing hours in Open-Meteo.
ciudad_real: 0 missing hours in Open-Meteo.
cordoba: 0 missing hours in Open-Meteo.
cuenca: 0 missing hours in Open-Meteo.
girona: 0 missing hours in Open-Meteo.
granada: 0 missing hours in Open-Meteo.
guadalajara: 0 missing hours in Open-Meteo.
guipuzcoa: 0 missing hours in Open-Meteo.
huelva: 0 missing hours in Open-Meteo.
huesca: 0 missing hours in Open-Meteo.
illes_balears: 0 missing hours in Open-Meteo.
jaen: 0 missing hours in Ope

# 6. Count rows per city

In [39]:
print("Rows per city in Open-Meteo:\n", df_openmeteo['city'].value_counts())
print("\nRows per city in PVGIS:\n", df_pvgis['city'].value_counts())

Rows per city in Open-Meteo:
 city
alava            140256
albacete         140256
alicante         140256
almeria          140256
asturias         140256
avila            140256
badajoz          140256
barcelona        140256
burgos           140256
caceres          140256
cadiz            140256
cantabria        140256
castellon        140256
ceuta            140256
ciudad_real      140256
cordoba          140256
cuenca           140256
girona           140256
granada          140256
guadalajara      140256
guipuzcoa        140256
huelva           140256
huesca           140256
illes_balears    140256
jaen             140256
la_coruna        140256
la_rioja         140256
las_palmas       140256
leon             140256
lleida           140256
lugo             140256
madrid           140256
malaga           140256
melilla          140256
murcia           140256
navarra          140256
ourense          140256
palencia         140256
pontevedra       140256
salamanca        140256
segov

# 7. Merge Open-Meteo and PVGIS datasets

In [40]:
df_merged = pd.merge(df_openmeteo, df_pvgis, on=["time", "city"], how="inner")

print("Merged dataset shape:", df_merged.shape)


Merged dataset shape: (7293312, 11)


# 8. Save processed datasets

In [43]:
# Define processed data path
processed_dir = root / 'data' / 'processed'
processed_dir.mkdir(parents=True, exist_ok=True)

# Export Open-Meteo clean dataset
df_openmeteo_clean = df_openmeteo.drop_duplicates(subset=["time", "city"])
df_openmeteo_clean.to_parquet(processed_dir / 'openmeteo_clean.parquet')
df_openmeteo_clean.to_csv(processed_dir / 'openmeteo_clean.csv', index=False)

# Export PVGIS clean dataset
df_pvgis_clean = df_pvgis.drop_duplicates(subset=["time", "city"])
df_pvgis_clean.to_parquet(processed_dir / 'pvgis_clean.parquet')
df_pvgis_clean.to_csv(processed_dir / 'pvgis_clean.csv', index=False)

# Export merged dataset
df_merged.to_parquet(processed_dir / 'merged_openmeteo_pvgis.parquet')
df_merged.to_csv(processed_dir / 'merged_openmeteo_pvgis.csv', index=False)

print("✅ Clean datasets saved as both Parquet and CSV in data/processed/")


✅ Clean datasets saved as both Parquet and CSV in data/processed/
